In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn.model_selection import cross_validate, StratifiedKFold
from collections import namedtuple
import itertools
import math

In [ ]:
n_cols = range(1, 65)
data = pd.read_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\final_dataset\\dataset.csv', usecols= n_cols)

In [ ]:
fraction = 0.1
dataset = data.sample(n=int(data.shape[0]*fraction), random_state=1)

In [ ]:
data_train = dataset.drop(['Y_CAT_STORNO_TOCH', 'DELTA_STORNO_FROMREZ',
                  'DELTA_STORNO_TOCH', 'STORNO_DA',  'SIF_HOTELA', 'OBRISANO_DA'], axis=1) 

In [ ]:
Ytrain = data_train.Y_CAT_STORNO_FROMREZ
Xtrain = data_train.drop('Y_CAT_STORNO_FROMREZ', axis=1)

In [ ]:
#params
LR = [0.1] # learning rate
EST = [5000] # n. estimators
DEPTH = [4] # max depth
res = []

In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=7)
scoring = ["precision_weighted", "recall_weighted", "f1_weighted"]
Result = namedtuple("Result", ["learning_rate", "estimators", "depth", "precision", "recall", "f1"])

In [ ]:
for l, e, d in itertool.product(LR, EST, DEPTH):
    model = XGBClassifier(learning_rate =l,
                          n_estimators=e,
                          max_depth=d,
                          subsample=1,
                          objective='multi:softprob',
                          seed=10,
                          verbosity = 2)
    #broj kategorija (potrebno za softmax)
    xgb_param = model.get_xgb_params()
    xgb_param['num_class'] = 7
    
    results = cross_validate(model, Xtrain, Ytrain, scoring=scoring, cv=10)
    
    r = Result(l, 
               e, 
               d, 
               results['precision_weighted'].mean(), 
               results['recall_weighted'].mean(), 
               results['f1_weighted'].mean())
    res.append(r)           


In [ ]:
data_exit = pd.DataFrame(res)
data_exit.to_csv('C:\\Users\\Aldo\\Documents\\fax\\Semestar 6\\Praksa\\final_dataset\\grid_search.csv')